In [1]:
import os
import sys
from pathlib import Path

# Add the project root to Python path
project_root = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
sys.path.insert(0, str(project_root))

# Change to project root directory
os.chdir(project_root)

In [2]:
from pathlib import Path

import numpy as np

# Clean imports from main package - everything needed is now available!
from EMFieldML import (
    FekoRunner,
    Learning,
    TargetDataBuilder,
    PolyCubeMaker,
    Visualize,
    config,
    paths,
    template,
    get_logger,
    GraphMaker,
    MakeYData,
    ResultMaker,
    TestMaker,
    Prediction
)

logger = get_logger(__name__)

# Genarate test data
This notebook creates test data for evaluating the predictions.

### Make parameter of test data

In [3]:
TestMaker.make_test_shield_coordinate()

In [ ]:
x_train_list = list(range(1,config.n_test_data+1))
output_path = paths.TEST_DATA_DIR / template.x_test
with open(output_path, "w") as f:
    for i in x_train_list:
        print(i, file=f)

logger.info(f"Saved {output_path}")

### Simulate test data
Simulate test data for getting the magnetic field data.

In [ ]:
x_train_list = list(range(1,config.n_test_data+1))

FekoRunner.run_simulation_test_list(x_train_list)

### Make STL of test data
Make STL file for making x_data

In [ ]:
FekoRunner.make_stl_test_list()

### Make x_data of test data
Make x_data. 

Model the test data and create a PolyCube mesh and an Exterior Grid.
However, this is created with the initial positions.

In [ ]:
PolyCubeMaker.make_modeling_all(
            n_shield_shape = config.n_test_data,
            path_stl_dir = paths.TEST_CIRCLE_STL_DIR,
            path_stl_file = template.test_circle_stl,
            path_polycube_dir = paths.TEST_CIRCLE_MOVE_X_DIR,
            path_polycube_file = template.test_circle_x,
            path_exterior_dir = paths.TEST_CIRCLE_PREDICTION_POINT_DIR,
            path_exterior_file = template.test_circle_prediction_point,
            )

To create the x_data from the PolyCube mesh, we incorporate the positional information.

In [ ]:

PolyCubeMaker.move_shield_polycube_all(
            n_shield_shape = config.n_test_data,
            path_polycube_dir = paths.TEST_CIRCLE_MOVE_X_DIR,
            path_polycube_file = template.test_circle_x,
            path_output_dir = paths.TEST_CIRCLE_MOVE_X_DIR,
            path_output_file = template.test_circle_move,
            )

Convert the initial Exterior Grid into an Exterior Grid that incorporates positional coordinates.

In [ ]:

for number in range(1,config.n_test_data+1):
    input_path_polycube = paths.TEST_CIRCLE_MOVE_X_DIR / template.test_circle_x.format(index=number)
    input_path_exterior = paths.TEST_CIRCLE_PREDICTION_POINT_DIR / template.test_circle_prediction_point.format(index=number)

    with open(paths.TEST_CIRCLE_MOVE_X_DIR / template.test_circle_move.format(index = number)) as f:
        l_strip = [float(s.rstrip()) for s in f.readlines()]

    PolyCubeMaker.move_shield_exterior(
        input_path_polycube = input_path_polycube,
        input_path_exterior = input_path_exterior,
        number = number,
        y = l_strip[-2],
        z = l_strip[-1],
        output_path_dir = paths.TEST_CIRCLE_PREDICTION_POINT_DIR,
        output_path_file = template.test_circle_prediction_point_move,
        )

### Make y_data of test data
Make y_data from Exterior Grid and simulated data.

In [ ]:
x_train_list = list(range(1,config.n_test_data+1))

TargetDataBuilder.make_new_ydatafile(
            x_train_list = x_train_list,
            output_dir = paths.TEST_Y_DATA_DIR,
            input_path_raw_data_dir = paths.TEST_CIRCLE_FEKO_RAW_DIR,
            input_path_prediction_point_dir = paths.TEST_CIRCLE_PREDICTION_POINT_DIR,
            input_path_prediction_point_file = template.test_circle_prediction_point_move,
            )

TargetDataBuilder.make_new_ydatafile_vector(
            x_train_list = x_train_list,
            output_dir = paths.TEST_Y_DATA_DIR,
            input_path_raw_data_dir = paths.TEST_CIRCLE_FEKO_RAW_DIR,
            input_path_prediction_point_dir = paths.TEST_CIRCLE_PREDICTION_POINT_DIR,
            input_path_prediction_point_file = template.test_circle_prediction_point_move,
            )

TargetDataBuilder.make_new_ydatafile_efficiency(
            x_train_list = x_train_list,
            output_dir = paths.TEST_Y_DATA_DIR,
            input_path_raw_data_dir = paths.TEST_CIRCLE_FEKO_RAW_DIR,
            )

## Make N change Result
Perform validation of the deviation when changing number of train data and the estimation error using test data.

In [ ]:
selected_prediction_point_list = config.selected_points

for number in selected_prediction_point_list:
    ResultMaker.make_N_change_deviation(number)
    ResultMaker.make_N_change_error(number)


In [ ]:

ResultMaker.sort_N_change_data()

In [ ]:
GraphMaker.N_change_deviation()

In [ ]:
GraphMaker.N_change_error()

In [ ]:
GraphMaker.Active_learning_map(paths.TRAIN_DIR / template.x_train_init, config.n_initial_train_data)
GraphMaker.Active_learning_map(paths.TRAIN_DIR / template.x_train_after_active_learning, config.n_train_data)


## Make cross validation result
Create a comparison of exterior grid and GPR optimization performed through cross validation.

### Make pattern for cross validation

In [ ]:
ResultMaker.cross_validation_pattern()

### Make y_data of each pattern

In [ ]:
for index in range(1,config.n_cross_validation+1):

    train_list_path = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = index)) / template.x_train
    test_list_path = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = index)) / template.x_test

    with open(train_list_path) as f:
        x_train_list = [int(s.rstrip()) for s in f.readlines()]

    with open(test_list_path) as f:
        x_test_list = [int(s.rstrip()) for s in f.readlines()]

    TargetDataBuilder.make_new_ydatafile(
                x_train_list = x_train_list,
                output_dir = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = index)) / paths.Y_DATA,
                prediction_point_list = list(range(config.n_prediction_points_level2)),
                n_prediction_points = config.n_prediction_points_level2,
                )

    TargetDataBuilder.make_new_ydatafile(
                x_train_list = x_test_list,
                output_dir = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = index)) / paths.TEST_Y_DATA,
                )

    TargetDataBuilder.make_new_ydatafile_vector(
                x_train_list = x_train_list,
                output_dir = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = index)) / paths.Y_DATA,
                )

    TargetDataBuilder.make_new_ydatafile_vector(
                x_train_list = x_test_list,
                output_dir = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = index)) / paths.TEST_Y_DATA,
                )

### Make fix grid y_data of each pattern

Make fixed prediction point. 

In [ ]:
ResultMaker.make_fix_prediction_point()

In [ ]:
for index in range(1,config.n_cross_validation+1):

    train_list_path = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = index)) / template.x_train
    test_list_path = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = index)) / template.x_test

    with open(train_list_path) as f:
        x_train_list = [int(s.rstrip()) for s in f.readlines()]

    with open(test_list_path) as f:
        x_test_list = [int(s.rstrip()) for s in f.readlines()]

    TargetDataBuilder.make_new_ydatafile(
                x_train_list = x_train_list,
                output_dir = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = index)) / paths.Y_DATA,
                input_path_prediction_point_dir = paths.CIRCLE_PREDICTION_POINT_DIR,
                input_path_prediction_point_file = template.result_prediction_point_move_fix,
                output_file = template.y_data_magnitude_fix,
                prediction_point_list = list(range(config.n_prediction_points_init)),
                n_prediction_points = config.n_prediction_points_init,
                )

    TargetDataBuilder.make_new_ydatafile(
                x_train_list = x_test_list,
                output_dir = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = index)) / paths.TEST_Y_DATA,
                input_path_prediction_point_dir = paths.CIRCLE_PREDICTION_POINT_DIR,
                input_path_prediction_point_file = template.result_prediction_point_move_fix,
                output_file = template.y_data_magnitude_fix,
                prediction_point_list = list(range(config.n_prediction_points_init)),
                n_prediction_points = config.n_prediction_points_init,
                )

    TargetDataBuilder.make_new_ydatafile_vector(
                x_train_list = x_train_list,
                output_dir = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = index)) / paths.Y_DATA,
                input_path_prediction_point_dir = paths.CIRCLE_PREDICTION_POINT_DIR,
                input_path_prediction_point_file = template.result_prediction_point_move_fix,
                output_file_theta = template.y_data_vector_theta_fix,
                output_file_phi = template.y_data_vector_phi_fix,
                prediction_point_list = list(range(config.n_prediction_points_init)),
                n_prediction_points = config.n_prediction_points_init,
                )

    TargetDataBuilder.make_new_ydatafile_vector(
                x_train_list = x_test_list,
                output_dir = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = index)) / paths.TEST_Y_DATA,
                input_path_prediction_point_dir = paths.CIRCLE_PREDICTION_POINT_DIR,
                input_path_prediction_point_file = template.result_prediction_point_move_fix,
                prediction_point_list = list(range(config.n_prediction_points_init)),
                n_prediction_points = config.n_prediction_points_init,
                )

    TargetDataBuilder.make_new_ydatafile_efficiency(
                x_train_list = x_train_list,
                output_dir = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = index)) / paths.Y_DATA,
                )

    TargetDataBuilder.make_new_ydatafile_efficiency(
                x_train_list = x_test_list,
                output_dir = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = index)) / paths.TEST_Y_DATA,
                )

### Make model

In [ ]:
for index in range(1,config.n_cross_validation+1):
    input_path_train_list = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = index)) / template.x_train
    x_train_data = Learning.make_x_train_data(input_path_train_list)

    Learning.learning_magneticfield(
                input_path_y_data = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = index)) / paths.Y_DATA / template.y_data_efficiency,
                output_path = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = index)) / template.model,
                x_train_data = x_train_data,
                lr_record = 0.02,
                iter_record = 1500,
                inter_record = 400,
                small_record = 10.0,
                )

    Learning.learning_magneticfield(
                input_path_y_data = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = index)) / paths.Y_DATA / template.y_data_efficiency,
                output_path = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = index)) / template.model_fix,
                x_train_data = x_train_data,
                lr_record = 0.0,
                iter_record = 0,
                inter_record = 0,
                small_record = 1.0,
                )


### Make prediction result

Make the magnetic field prediction result of adaptive exterior grid.

In [ ]:
for pattern in range(1,config.n_cross_validation+1):
    Prediction.pre_magneticfield(
            model_path = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / template.model_fix,
            input_path_train_list = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / template.x_train,
            input_path_test_list = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / template.x_test,
            input_path_y_train_data = str(Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / paths.Y_DATA / template.y_data_magnitude),
            input_path_y_test_data = str(Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / paths.TEST_Y_DATA / template.y_data_magnitude),
            output_path = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / template.result_prediction_magnitude_move,
            func_calculate_error = Prediction.calculate_error_square,
            )

Make the magnetic field prediction result of fix prediction point.

In [ ]:
for pattern in range(1,config.n_cross_validation+1):
    Prediction.pre_magneticfield(
            model_path = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / template.model_fix,
            input_path_train_list = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / template.x_train,
            input_path_test_list = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / template.x_test,
            input_path_y_train_data = str(Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / paths.Y_DATA / template.y_data_magnitude_fix),
            input_path_y_test_data = str(Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / paths.TEST_Y_DATA / template.y_data_magnitude_fix),
            output_path = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / template.result_prediction_magnitude_no_move,
            func_calculate_error = Prediction.calculate_error_square,
            n_point = config.n_prediction_points_init,
            )

Make the vector prediction result of adaptive exterior grid.

In [ ]:
for pattern in range(1,config.n_cross_validation+1):
    Prediction.pre_vector(
        model_path = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / template.model_fix,
        input_path_train_list = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / template.x_train,
        input_path_test_list = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / template.x_test,
        input_path_y_train_data_phi = str(Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / paths.Y_DATA / template.y_data_vector_phi),
        input_path_y_train_data_theta = str(Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / paths.Y_DATA / template.y_data_vector_theta),
        input_path_y_test_data_phi = str(Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / paths.TEST_Y_DATA / template.y_data_vector_phi),
        input_path_y_test_data_theta = str(Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / paths.TEST_Y_DATA / template.y_data_vector_theta),
        output_path = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / template.result_prediction_vector_move,
        n_point = config.n_prediction_points_level1,
        )

Make the vector prediction result of fix prediction point.

In [ ]:
for pattern in range(1,config.n_cross_validation+1):
    Prediction.pre_vector(
        model_path = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / template.model_fix,
        input_path_train_list = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / template.x_train,
        input_path_test_list = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / template.x_test,
        input_path_y_train_data_phi = str(Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / paths.Y_DATA / template.y_data_vector_phi_fix),
        input_path_y_train_data_theta = str(Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / paths.Y_DATA / template.y_data_vector_theta_fix),
        input_path_y_test_data_phi = str(Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / paths.TEST_Y_DATA / template.y_data_vector_phi_fix),
        input_path_y_test_data_theta = str(Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / paths.TEST_Y_DATA / template.y_data_vector_theta_fix),
        output_path = Path(str(paths.RESULT_CROSS_PATTERN_DIR).format(index = pattern)) / template.result_prediction_vector_no_move,
        n_point = config.n_prediction_points_init,
        )

Combine the results of all patterns into a single file.

In [ ]:
ResultMaker.sort_list(
    input_path_dir = paths.RESULT_CROSS_PATTERN_DIR,
    input_path_file = template.result_prediction_magnitude_move,
    output_path = paths.RESULT_CROSS_DIR / template.result_prediction_magnitude_move,
)

ResultMaker.sort_list(
    input_path_dir = paths.RESULT_CROSS_PATTERN_DIR,
    input_path_file = template.result_prediction_magnitude_no_move,
    output_path = paths.RESULT_CROSS_DIR / template.result_prediction_magnitude_no_move,
)

ResultMaker.sort_list(
    input_path_dir = paths.RESULT_CROSS_PATTERN_DIR,
    input_path_file = template.result_prediction_vector_move,
    output_path = paths.RESULT_CROSS_DIR / template.result_prediction_vector_move,
)

ResultMaker.sort_list(
    input_path_dir = paths.RESULT_CROSS_PATTERN_DIR,
    input_path_file = template.result_prediction_vector_no_move,
    output_path = paths.RESULT_CROSS_DIR / template.result_prediction_vector_no_move,
)

Make graph of comparison magnitude and vector result.

In [ ]:
GraphMaker.magnitude_adaptive()

In [ ]:
GraphMaker.vector_adaptive()

## Make full data set result

### Change postprocessing value and measure the error rate at each value
Efficiency

In [ ]:
for postprocessing in range(1,11):
    Prediction.pre_magneticfield(
        model_path = paths.LEARNING_MODEL_DIR / template.model,
        input_path_train_list = paths.TRAIN_DIR / template.x_train_after_active_learning,
        input_path_test_list = paths.TEST_DATA_DIR / template.x_test,
        input_path_y_train_data = str(paths.Y_DATA_DIR / template.y_data_efficiency),
        input_path_y_test_data = str(paths.TEST_Y_DATA_DIR / template.y_data_efficiency),
        output_path = paths.RESULT_FULLDATA / template.result_prediction_efficiency_post_under.format(index = postprocessing),
        func_calculate_error = Prediction.calculate_error,
        n_point = 1,
        input_path_circle_test = str(paths.TEST_CIRCLE_MOVE_X_DIR / template.test_circle_move),
        postprocessing_value = 1 / postprocessing,
        noise_scale_change = True,
        )

for postprocessing in range(1,11):
    Prediction.pre_magneticfield(
        model_path = paths.LEARNING_MODEL_DIR / template.model,
        input_path_train_list = paths.TRAIN_DIR / template.x_train_after_active_learning,
        input_path_test_list = paths.TEST_DATA_DIR / template.x_test,
        input_path_y_train_data = str(paths.Y_DATA_DIR / template.y_data_efficiency),
        input_path_y_test_data = str(paths.TEST_Y_DATA_DIR / template.y_data_efficiency),
        output_path = paths.RESULT_FULLDATA / template.result_prediction_efficiency_post.format(index = postprocessing),
        func_calculate_error = Prediction.calculate_error,
        n_point = 1,
        input_path_circle_test = str(paths.TEST_CIRCLE_MOVE_X_DIR / template.test_circle_move),
        postprocessing_value = postprocessing,
        noise_scale_change = True,
        )

Magnitude of magnetic field

In [ ]:
for postprocessing in range(1,21):
    Prediction.pre_magneticfield(
        model_path = paths.LEARNING_MODEL_DIR / template.model,
        input_path_train_list = paths.TRAIN_DIR / template.x_train_after_active_learning,
        input_path_test_list = paths.TEST_DATA_DIR / template.x_test,
        input_path_y_train_data = str(paths.Y_DATA_DIR / template.y_data_magnitude),
        input_path_y_test_data = str(paths.TEST_Y_DATA_DIR / template.y_data_magnitude),
        output_path = paths.RESULT_FULLDATA / template.result_prediction_magnitude_post.format(index = postprocessing),
        func_calculate_error = Prediction.calculate_error,
        n_point = config.n_prediction_points_level1,
        input_path_circle_test = str(paths.TEST_CIRCLE_MOVE_X_DIR / template.test_circle_move),
        postprocessing_value = postprocessing,
        noise_scale_change = True,
        )

Vector of magnetic field

In [ ]:
for postprocessing in range(1,11):
    Prediction.pre_vector(
        model_path = paths.LEARNING_MODEL_DIR / template.model,
        input_path_train_list = paths.TRAIN_DIR / template.x_train_after_active_learning,
        input_path_test_list = paths.TEST_DATA_DIR / template.x_test,
        input_path_y_train_data_phi = str(paths.Y_DATA_DIR / template.y_data_vector_phi),
        input_path_y_train_data_theta = str(paths.Y_DATA_DIR / template.y_data_vector_theta),
        input_path_y_test_data_phi = str(paths.TEST_Y_DATA_DIR / template.y_data_vector_phi),
        input_path_y_test_data_theta = str(paths.TEST_Y_DATA_DIR / template.y_data_vector_theta),
        output_path = paths.RESULT_FULLDATA / template.result_prediction_vector_post_under.format(index = postprocessing),
        n_point = config.n_prediction_points_level1,
        input_path_circle_test = str(paths.TEST_CIRCLE_MOVE_X_DIR / template.test_circle_move),
        postprocessing_value = 1 / postprocessing,
        noise_scale_change = True,
        )

    Prediction.pre_vector(
        model_path = paths.LEARNING_MODEL_DIR / template.model,
        input_path_train_list = paths.TRAIN_DIR / template.x_train_after_active_learning,
        input_path_test_list = paths.TEST_DATA_DIR / template.x_test,
        input_path_y_train_data_phi = str(paths.Y_DATA_DIR / template.y_data_vector_phi),
        input_path_y_train_data_theta = str(paths.Y_DATA_DIR / template.y_data_vector_theta),
        input_path_y_test_data_phi = str(paths.TEST_Y_DATA_DIR / template.y_data_vector_phi),
        input_path_y_test_data_theta = str(paths.TEST_Y_DATA_DIR / template.y_data_vector_theta),
        output_path = paths.RESULT_FULLDATA / template.result_prediction_vector_post.format(index = postprocessing),
        n_point = config.n_prediction_points_level1,
        input_path_circle_test = str(paths.TEST_CIRCLE_MOVE_X_DIR / template.test_circle_move),
        postprocessing_value = postprocessing,
        noise_scale_change = True,
        )

### Make graph

Result of changing postprocessing value

In [ ]:
GraphMaker.magnitude_postprocessing_compare()

In [ ]:
GraphMaker.efficiency_postprocessing_compare()

Error map at optimum postprocessing value

In [ ]:
GraphMaker.error_map(
            input_path = paths.RESULT_FULLDATA / template.result_prediction_magnitude_post.format(index = 20),
            type_error = "magnitude",
            vmax = 0.20,
            )

In [ ]:
GraphMaker.error_map(
            input_path = paths.RESULT_FULLDATA / template.result_prediction_efficiency_post_under.format(index = 6),
            type_error = "efficiency",
            vmax = 0.04,
            )

Error map at no postprocessing value

In [ ]:
GraphMaker.error_map(
            input_path = paths.RESULT_FULLDATA / template.result_prediction_magnitude_post.format(index = 1),
            type_error = "magnitude",
            vmax = 1.0,
            cmap = "RdYlBu_r",
            )
GraphMaker.error_map(
            input_path = paths.RESULT_FULLDATA / template.result_prediction_efficiency_post_under.format(index = 1),
            type_error = "efficiency",
            vmax = 0.08,
            cmap = "RdYlBu_r",
            )
GraphMaker.error_map(
            input_path = paths.RESULT_FULLDATA / template.result_prediction_vector_post_under.format(index = 1),
            type_error = "vector",
            vmax = 100,
            cmap = "RdYlBu_r",
            )

## Make magnitude map

### Make simulation result
Make ferrite shield parameter.

In [ ]:
MakeYData.make_ferrite_parameter()

Make prediction point.

In [ ]:
MakeYData.make_prediction_point()

Do FEKO and get the simulated result.

In [ ]:
for number in range(625):
    FekoRunner.run_solver(
        number = number,
        path_save_dir = paths.RESULT_MAGNITUDE_MAP / "circle_raw_data",
        path_circle_dir = paths.RESULT_MAGNITUDE_MAP / "circle_parameter",
        path_circle_file = template.circle_move_param,
        )

Make y_data.

In [ ]:
TargetDataBuilder.make_new_ydatafile(
            x_train_list = list(range(625)),
            output_dir = paths.RESULT_MAGNITUDE_MAP / paths.Y_DATA,
            input_path_raw_data_dir = paths.RESULT_MAGNITUDE_MAP / "circle_raw_data",
            input_path_prediction_point_dir = paths.RESULT_MAGNITUDE_MAP / "circle_prediction_point",
            input_path_prediction_point_file = template.circle_prediction_point_move,
            prediction_point_list = [0],
            n_prediction_points = 1,
            )

TargetDataBuilder.make_new_ydatafile_vector(
            x_train_list = list(range(625)),
            output_dir = paths.RESULT_MAGNITUDE_MAP / paths.Y_DATA,
            input_path_raw_data_dir = paths.RESULT_MAGNITUDE_MAP / "circle_raw_data",
            input_path_prediction_point_dir = paths.RESULT_MAGNITUDE_MAP / "circle_prediction_point",
            input_path_prediction_point_file = template.circle_prediction_point_move,
            prediction_point_list = [0],
            n_prediction_points = 1,
            )

TargetDataBuilder.make_new_ydatafile_efficiency(
            x_train_list = list(range(625)),
            output_dir = paths.RESULT_MAGNITUDE_MAP / paths.Y_DATA,
            input_path_raw_data_dir = paths.RESULT_MAGNITUDE_MAP / "circle_raw_data",
            )

### Make graph

In [ ]:
GraphMaker.magnitude_map_magnetic(1)
GraphMaker.magnitude_map_magnetic(20)
GraphMaker.magnitude_map_efficiency(1)
GraphMaker.magnitude_map_efficiency(1/6)


## Time measurement

Measure simulation time.

In [ ]:
ResultMaker.measure_simulation_time()

Measure this method time.

In [ ]:
shield_number = 90
Visualize = Visualize()
Visualize.visualize(shield_number, measure_time_mode = True)

## Compare results by ROI and movement

In [ ]:
GraphMaker.changeEMfieldByRoiAndMovement(np.array([0.0,-0.15,0.36]), relativeMove = True)
# Max value:  12.750459400893504
# Min value:  4.340637760946816

In [ ]:
GraphMaker.changeEMfieldByRoiAndMovement(np.array([0.0, -0.245, 0.12]))
# Max value:  16.560321960968945
# Min value:  5.469208228532827

In [ ]:
GraphMaker.changeEfficiencyByRoiAndMovement()

In [ ]:
GraphMaker.changeEfficiencyByDoubleRoi()